# Justificativa para o modelo de classificaçao

-Beleza! O recomendador parece estar funcionar, mas como posso saber se ele está funcionando bem? A avaliação de um sistema de recomendação pode ser bastante complicada. A métrica de ouro para um sistema de recomendação é o quanto o sistema agregará valor ao usuário e ao negócio. Por fim, podemos realizar um teste A / B para ver se as recomendações ajudaram o usuário a encontrar o imóvel de uma forma mais fluída se comparado às técnicas tradicionais.

Mas existem outras técnicas para avaliar um sistema de recomendação, ou seja, distinguir as recomendações boas das ruins. No caso binário, isso é feito indicando “1” como sendo uma boa recomendação, enquanto “0” significa uma recomendação ruim. Após realizada suficiente interação do usuário com o sistema de recomendação, e de posse dos dados com seus respectivos labels (target, ou seja, “0” ou ”1”),  podemos treinar um modelo de classificação nos dados que possuem label e testar nos dados que não possuem label, retornando um ranking (predict proba) com os anúncios que mais se aproximam daqueles classificados como “1”. Assim, ao mesmo tempo que podemos avaliar se o sistema está funcionando como previsto, conseguimos realizar uma espécie de “curadoria” nos anúncios que o usuário provavelmente não veria por estar em clusters diferentes e/ou por conta do enorme volume de dados.

In [10]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
import random
from random import shuffle
import webbrowser

import json
import tqdm
import os

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import roc_auc_score, average_precision_score, precision_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import category_encoders as ce # Target encoder
from sklearn.preprocessing import LabelEncoder, StandardScaler, MaxAbsScaler
from lightgbm import LGBMClassifier
from skopt import dummy_minimize, gp_minimize, forest_minimize
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import VotingClassifier
import scipy
import matplotlib.pyplot as plt
import spacy
import nltk 
from nltk import tokenize
from string import punctuation

import warnings
warnings.filterwarnings("ignore")

%config Completer.use_jedi = False
sns.set()
plt.style.use('seaborn-whitegrid')
plt.rcParams['axes.grid'] = True

In [11]:
path = './data/04_feat_eng_df/'

# Nome dos arquivos dentro do diretorio acima (raw_df)
filenames = os.listdir(path)

# Dataframe dos dados sem tratamento
df = pd.read_csv(path + filenames[-1], sep='|') # -1 para pegar o ultimo dataframe adquirido

In [12]:
# Dropando colunas antes do active learning
# cols = ['list_id', 'title', 'logradouro', 'latitude', \
#         'longitude', 'zipcode', 'source_x', 'desc_full', 'altitude', 'sellername', 'state', 'region', 'complemento']

df_mod = df.drop('desc_full', axis=1) # Dataframe modificado
desc = df['desc_full'] # Series com a descricao de cada ap para o tfidf

In [13]:
df.columns

Index(['list_id', 'link', 'price', 'condominio', 'rooms', 'bathrooms',
       'garage_spaces', 'size', 'pictures', 'desc_full', 'padrao_bairro',
       'price_total', 'barra funda', 'bela vista', 'belenzinho', 'bom retiro',
       'cambuci', 'campos eliseos', 'centro', 'consolacao', 'liberdade', 'luz',
       'morro dos ingleses', 'parque industrial tomas edson', 'republica',
       'santa cecilia', 'santa efigenia', 'se', 'varzea da barra funda',
       'vila buarque', 'vila deodoro'],
      dtype='object')

# Active Learning

In [14]:
df_mod.columns

Index(['list_id', 'link', 'price', 'condominio', 'rooms', 'bathrooms',
       'garage_spaces', 'size', 'pictures', 'padrao_bairro', 'price_total',
       'barra funda', 'bela vista', 'belenzinho', 'bom retiro', 'cambuci',
       'campos eliseos', 'centro', 'consolacao', 'liberdade', 'luz',
       'morro dos ingleses', 'parque industrial tomas edson', 'republica',
       'santa cecilia', 'santa efigenia', 'se', 'varzea da barra funda',
       'vila buarque', 'vila deodoro'],
      dtype='object')

## Modelo Naive para Labeling

In [15]:
model_labeling = RandomForestClassifier(n_estimators=200, 
                                  random_state=0,
                                  min_samples_leaf=1,
                                  class_weight='balanced', 
                                  n_jobs=-1)

## Funcoes para a otimização bayesiana

In [16]:
def tuning_rf(params):
    """ Função para tunar o modelo do random forest"""
    bootstrap = params[0]
    max_depth = params[1]
    max_features = params[2] 
    min_samples_leaf = params[3] 
    min_samples_split = params[4]
    n_estimators = params[5]
    
    model = RandomForestClassifier(bootstrap=bootstrap, max_depth=max_depth, max_features=max_features,
                          min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, n_estimators=n_estimators)
    
    model.fit(X, y)
    y_pred = model.predict_proba(X)[: ,1]  # y_pred entre treino e teste
      
    return -average_precision_score(y, y_pred)

space_rf = [[True, False],
            [1, 2, 5, 10, None],
            ['auto', 'sqrt'], 
            (1, 4), 
            (2, 10),
            (500, 1000)]
##################################################################################################################

def tuning_lr(params):
    """ Função para tunar o modelo logistic regression """
    solver = params[0]
    penalty = params[1]
    C = params[2]
    
    model = LogisticRegression(solver=solver, penalty=penalty, C=C)
    
    model.fit(X, y)
    y_pred = model.predict_proba(X)[: ,1]  # y_pred entre treino e teste
    
    return -average_precision_score(y, y_pred)


space_lr = [['newton-cg', 'lbfgs', 'liblinear'],
            ['l2'],
            (0.01, 100)]

## função para o Active learning

In [17]:
def active_learning(df):
    """ return to_label, top_itens"""

    df = df.drop(['link'], axis=1) # Delerando o link antes de treinar o modelo
    df_model = df[df['label'].notnull()] # Selecionando os dados com label para treinar o modelo
    
    # Se houver mais de 10 itens com label &
    # Se todos os itens nao forem iguais a 0, treinar o modelo
    if (len(df_model)>= 10):
        #(0 is not df_model.value_counts().to_list()):
        #(len(df_model[df_model['label'] == 1] >= 2)): 
           
        X = df_model.drop('label', axis=1) # Features
        y = df_model['label'].astype(int) # Target
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=0) # Split entre treino e teste
        
        print('========== Informações do treinamento ==========')
        print(f'{round(X_train.shape[0] / df.shape[0], 2)*100}% dos dados anotados, sendo que {round(y_train.mean(), 2) * 100}% desses dados sao positivos')
        
        model_labeling.fit(X_train, y_train) # Fitando entre treino e teste (pipeline para labeling)
        y_pred = model_labeling.predict_proba(X_test)[: ,1]  # y_pred entre treino e teste (pipeline para laberling)
        try:
            print(f'average precision score: {average_precision_score(y_test, y_pred)}') # Precision
            print(f'roc auc score: {roc_auc_score(y_test, y_pred)}') # ROC
        except:
            print('Only one class present in y_true. ROC AUC score is not defined in that case.')
        
        ################################# Selecao dos itens para label ##########################################
        
        df_unlabeled = df[df['label'].isnull()] # Data sem label
        X_unlabeled = df_unlabeled.drop('label', axis=1) # X sem label (dropando a feature label)
        
        y_pred_unlabeled = model_labeling.predict_proba(X_unlabeled)[:, 1] # y_pred para os dados sem label
        df_unlabeled['y_pred'] = y_pred_unlabeled # Juntando a o y_pred sem label com o dataframe sem label
        
        top_itens = df_unlabeled.sort_values(by='y_pred', ascending=False)[:20].index # Top 20 dos dados do dataframe sem label
        to_label_undecided = df_unlabeled[(df_unlabeled['y_pred'] >= 0.50) & (df_unlabeled['y_pred'] < 1)] # Item para labeling (em torno de 50%)

        if len(to_label_undecided) < 7: # Se Houver dados insuficientes para gerar pelo menos 7 valores em torno de 0.4 e 0.6
            to_label = list(df_unlabeled.sample(5).index) # Adquire 5 valores aleatorios
        else:
            to_label = list(pd.concat([to_label_undecided.sample(7),  # Adquiri 7 valores indecisos
                                       df_unlabeled.sample(3)]).index) # Adquiri 3 valores aleatorios

        print('\n========= y_pred do item para treino ==========')
        print(df_unlabeled.loc[to_label]['y_pred']) # y_pred do dado sorteado
        
        print('\n========= Feature Importance ==========') # Feature Importance
        feat_importance = pd.DataFrame(zip(X_train.columns, model_labeling.feature_importances_), columns=['feature', 'importance'])
        feat_importance = feat_importance.set_index('feature').sort_values('importance', ascending=False)
        print(feat_importance[:5]) # 5 features mais importantes
       
    else: # Se houver menos de 10 itens com label
        print('Ps: Itens Aleatórios')
        to_label = list(df.sample(10).index) # Sorteia numeros aleatório entre os indices sem label
        top_itens = to_label
        
    shuffle(to_label) # Embaralha os itens para treino
    
    return to_label, top_itens

## Função para o labeling

In [18]:
def get_label(df, print_recommendation=False):
    """ Funcao para inserir o label 0 e 1 no dataframe """
    
    df_temp = df.copy()# Lista para guardar os indices dos itens
    if {'label'}.issubset(df_temp.columns) == False: # Verifica se há a coluna 'label'
        df_temp['label'] = np.nan # Se nao hover cria uma coluna 'label'
    
    while True:
        
        to_label, top_itens = active_learning(df_temp) # Adquirindo o dados para o active learning e os top itens
        
        ##############################################################################        
        # Printa os X melhores predic probas adquiridos pela funcao 'active_learning'
        if print_recommendation==True:
            print('\n  ========== Recomendação Inicial ==========')
            for i in top_itens:
                	print(df_temp.loc[i]['link'])
        ###############################################################################

        for n in to_label: # Looping para os itens selecionados para labeling
            
            ape = df_temp.loc[n] # Seleciona ape sorteado
            
            print('\n ========== Dados do item ======== \n', ape['link']) # Imprime o link
            print(ape, '\n') # Imprime as caracteristicas dropando as colunas que contem apenas zero geradas pelo
            print('='*80)
            webbrowser.open(ape['link']) # Abre o link
            
            n_labels = df_temp[~df_temp['label'].isnull()].shape[0] # Quantidade de itens com label
            print(f'Quantidade de itens com label: {n_labels}') 
            
            ## 1a condicional (0 ou 1)
            while True:
                inp = str(input('Label: 0 ou 1')) # Adquire o label do items selecionado
                
                if (inp == '0') | (inp == '1'): # Verifica se o item é 0 ou 1
                    df_temp['label'].loc[int(n)] = inp # Insere o label no item selecionado
                    break
                else:
                    continue
                    
            df_temp.to_csv(f'./data/05_labeled_df/df_w_label.csv', sep='|',  index=False) # Salva o dataframe
            
            ## 2a condicional (continuar? s ou n)
            while True:
                inp = str(input('Continuar? S ou N')).lower() # Le a resposta
                if (inp == 's') | (inp == 'n'): # Se a resposta for 'sim' ou 'não'
                    if inp == 'n': # Condicional para saber se nao continuoa com o lopping
                        n_labels = df_temp[~df_temp['label'].isnull()].shape[0] # Quantidade de labels no dataframe
                        print(f'Quantidade de itens com label: {n_labels}') # Printa a quantidade de itens com label

                        return df_temp # Retorna o dataframe + label

                    else:
                        break

## Função de Preprocessamento/Vetorização de texto/Clusterização

In [22]:
%%time
def nlp_trat(dataset):
    """ Funçao para a criacao da matriz esparça do tfidf"""
    tfidf = TfidfVectorizer(lowercase=True, max_features=50, min_df=2, ngram_range=(1, 2))
    token_espaco = tokenize.WhitespaceTokenizer()
    palavras_irrelevantes = nltk.corpus.stopwords.words("portuguese")
    
    stemmer = nltk.RSLPStemmer()
    token_pontuacao = tokenize.WordPunctTokenizer()
    
    # Criação de lista com pontuação
    pontuacao = list()
    for ponto in punctuation:
        pontuacao.append(ponto)
    pontuacao
    
    # Criação de lista complementar de palavras para remover do dataset (justificativas no notebook de storytelling)
    compl_stop_words = ['dormitorio','sao','paulo','apartamento','apto','sp','r$','m2','alugar','aluga','dormitorios',
                        'aluguel','locacao','para','com','em','por','mes','r', 'codigo', 'anuncio']
    
    # Lista final de palavras e caracteres para serem removiodos
    pontuacao_stop_words = pontuacao + palavras_irrelevantes + compl_stop_words
    frase_processada = list()
    nova_frase = list()
    

    
    for desc in dataset:
        palavras_texto = token_pontuacao.tokenize(desc)
        for palavra in palavras_texto:
            if palavra not in pontuacao_stop_words and palavra.isalpha():
                nova_frase.append(stemmer.stem(palavra))
        frase_processada.append(' '.join(nova_frase))
    frase_final = frase_processada
    
# TFIDF
    tfidf_bruto = tfidf.fit_transform(frase_final)
    
    return tfidf_bruto

sparsed_df = nlp_trat(desc) # TF-IDF da descricao completa
scipy.sparse.save_npz('./tmp/sparsed_df.npz', sparsed_df) # Salvando a matrix espar;

In [31]:
scipy.sparse.load_npz('./tmp/sparsed_df.npz')

<2110x50 sparse matrix of type '<class 'numpy.float64'>'
	with 105248 stored elements in Compressed Sparse Row format>

***

# Rodando o modelo

In [ ]:
%%time
# Função para labeling
df_mod = get_label(df_mod, print_recommendation=True)

# Preparando os dados com o dataframe labeled
df_labeled = df_mod[df_mod['label'].notnull()] # Selecionando os dados com label para treinar o modelo
X_numeric = df_labeled.drop(['label', 'link'], axis=1) # X para tunnin com dados numericos
X_tfidf = sparsed_df.tocsr()[df_labeled.index] # X para tunning com tfidf
print(desc.loc[df_labeled.index])
y = df_labeled['label'].astype(int) # Target
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=0) # Split entre treino e teste

# Fazendo o predict do modelo com os dados sem label
print('\n Aguarde um momento .......... (vetorização das variáveis categóricas)\n') # O treino pode levar algums minutos, dependendo do tamanho do dataframe
df_unlabeled = df_mod[df_mod['label'].isnull()] # dataframe sem label
X_unlabeled_numeric = df_unlabeled.drop(['link', 'label'], axis=1) # Features sem label para treino dados numericos
X_unlabeled_tfidf= sparsed_df.tocsr()[df_unlabeled.index] # Features sem label Fazendo a vetorizacao dos dados desc

for i in zip([X_numeric, X_tfidf], [X_unlabeled_numeric, X_unlabeled_tfidf], ['Recomendacao final', 'Tente dar uma olhada nestes tb 🙂']):
    
    X = i[0]
    X_unlabeled =i[1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=0) # Split entre treino e teste
    
    print('\n Aguarde um momento .......... (tunando o modelo)\n') # O treino pode levar algums minutos, dependendo do tamanho do dataframe
    
    ''' ============== Bayesian Optimization and Hyperparameter Tuning ================ '''
    
    ''' ================================ RANDOM FOREST ================================ '''
    
    # Tunando modelo random forest
    result = forest_minimize(tuning_rf, space_rf, random_state=0, n_random_starts=20, n_calls=30, verbose=False)
    
    bootstrap = result.x[0]
    max_depth = result.x[1]
    max_features = result.x[2]
    min_samples_leaf = result.x[3] 
    min_samples_split = result.x[4] 
    n_estimators = result.x[5]
    
    # Modelo Random Forest
    rf = RandomForestClassifier(bootstrap=bootstrap, max_depth=max_depth, max_features=max_features,
                                min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, n_estimators=n_estimators,
                                class_weight='balanced', random_state=0, n_jobs=-1)
    
    
    ''' ================================ LOGISTIC REGRESSION ================================ '''
        
     # Tunando modelo Logistic Regression
    result = forest_minimize(tuning_lr, space_lr, random_state=0, n_random_starts=20, n_calls=30, verbose=False)
    
    solver = result.x[0]
    penalty = result.x[1]
    C = result.x[2]
    
    # Modelo Logistic Regression
    lr = LogisticRegression(solver=solver, penalty=penalty, C=C)

    ''' ================================ ENSEMBLE VOTING ================================ '''
    
    # instanciação do Ensemble
    voting_model = VotingClassifier(estimators = [('random forest', rf),
                                                  ('logistic regression', lr)],
                                                voting = 'soft')
    
    print('\n Aguarde um momento .......... (treinando o modelo)\n') # O treino pode levar algums minutos, dependendo do tamanho do dataframe
    
    voting_model.fit(X_train, y_train)
    y_pred = voting_model.predict_proba(X_test)[: ,1]  # y_pred entre treino e teste
    try:
        print(f'average precision score: {average_precision_score(y_test, y_pred)}') # Precision
        print(f'roc auc score: {roc_auc_score(y_test, y_pred)}') # ROC
        
    except:
        print('Only one class present in y_true. ROC AUC score is not defined in that case.')
    
    df_unlabeled['y_pred'] = voting_model.predict_proba(X_unlabeled)[:, 1]# Predicts proba para os dados sem label
    final_rec_top_itens = df_unlabeled.sort_values('y_pred', ascending=False)[:20].index # indices dos top_itens
    
    print(f'\n ========== {i[2]} ==========')
    for i in df_mod.loc[final_rec_top_itens]['link']:
            print(i)
            
df_mod.to_csv(f'./data/05_labeled_df/df_w_label.csv', sep='|',  index=False) # Cria o dataframefeat_importance

<img src="img/app2.jpg" width="1200" />